# Chapter 16. Coroutines

# Basic Behavior of a Generator Used as a Coroutine

In [3]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)

In [4]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x11029fe58>

In [5]:
next(my_coro)

-> coroutine started


In [6]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

`inspect.getgeneratorstate()`: Inspect the state of coroutine
- `GEN_CREATED`
- `GEN_RUNNING`
- `GEN_SUSPENDED`
    - waiting at `yield`
- `GEN_CLOSED`

In [8]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [9]:
my_coro2 = simple_coro2(14)
from inspect import getgeneratorstate
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [10]:
next(my_coro2)

-> Started: a = 14


14

In [11]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [12]:
my_coro2.send(28)

-> Received: b = 28


42

In [13]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [15]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

# Example: Coroutine to Compute a Running Average

In [22]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [23]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)

10.0

In [24]:
coro_avg.send(30)

20.0

In [25]:
coro_avg.send(5)

15.0

In [26]:
from functools import wraps

def coroutine(func):
    """Decorator: primes `func` by advancing to first `yield`"""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [28]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [29]:
coro_avg = averager()
from inspect import getgeneratorstate
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [30]:
coro_avg.send(10)

10.0

In [31]:
coro_avg.send(30)

20.0

In [32]:
coro_avg.send(5)

15.0

# Coroutine Termination and Exception Handling

In [40]:
class DemoException(Exception):
    """An exception type for the demostration."""


def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
    else:
        print('-> coroutine received: {!r}'.format(x))
        raise RuntimeError('This line should never run.')

In [41]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [42]:
exc_coro.send(11)

In [43]:
exc_coro.send(22)

In [44]:
exc_coro.close()

In [45]:
from inspect import getgeneratorstate
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [49]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [50]:
exc_coro.send(11)

In [51]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [52]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [53]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [54]:
exc_coro.send(11)

In [55]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [56]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [57]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_finallY():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

# Returning a Value from a Coroutine

In [59]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

In [60]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

In [61]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
result

Result(count=3, average=15.5)

In [62]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

# the subgenerator
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

# the delegating generator
def grouper(results, key):
    while True:
        results[key] = yield from averager()

# client code
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
    report(results)

def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(result.count, group, result.average, unit))

data = {
    'girls;kg': [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m': [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg': [39.0, 40.8, 43.2, 40.8, 43.1, 30.6, 41.4, 40.6, 36.3],
    'boys;m': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

 9 boys  averaging 39.53kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m
